# Data Extraction from Twitter and Reddit APIs

This notebook deals with extracting data from twitter and Reddit APIs for Authors, Books and Publishers

## Reddit

In [8]:
import praw                                  #import the required packages.
import time
import pandas as pd
import tweepy
import re
import json
import csv
reddit = praw.Reddit(client_id='Yun9nTCW2H42NA',    # initializing the reddit vsriable with the reddit credentials.
                     client_secret='q0KfyRpdZ03J55_ljncf9Q-PtXY',
                     password='Reddit@18101993',
                     user_agent='ABC',
                     username='TinyTeddybear77')

In [3]:
X1 = pd.read_csv("TOTAL.csv")  #reading the .csv file with the names of the books so that they can be extracted from the reddit API.

In [4]:
X1.head()        #head of the dataframe read by the file

,Unnamed: 0,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,...,Setting,Awards,Unnamed: 0.1,Author Name,Works Count,Gender,About,Born Date,Died Date,Publications Influenced
0,0,9,960,960,3338963,311,1416524797,9.781417e+12,Dan Brown,2000.0,...,"['Rome', 'Vatican City', 'Italy']","[""Humo's Gouden Bladwijzer (2005)""]",483,Dan Brown,85,male,Dan Brown is the author of numerous #1 bestsel...,Unknown,Alive,Unknown
1,1,145,976,976,3135896,180,671027387,9.780671e+12,Dan Brown,2001.0,...,"['Arctic Circle', 'Washington D.C.']",NaN,483,Dan Brown,85,male,Dan Brown is the author of numerous #1 bestsel...,Unknown,Alive,Unknown
2,2,201,6411961,6411961,6600281,208,385504225,9.780386e+12,Dan Brown,2009.0,...,"['Washington, D.C.']",['Goodreads Choice Award Nominee for Favorite ...,483,Dan Brown,85,male,Dan Brown is the author of numerous #1 bestsel...,Unknown,Alive,Unknown
3,3,11,77203,77203,3295919,283,1594480001,9.781594e+12,Khaled Hosseini,2003.0,...,"['Kabul', 'Fremont, California', 'Afghanistan'...",['Borders Original Voices Award for Fiction (2...,430,Khaled Hosseini,22,male,"Hosseini was born in Kabul, Afghanistan, in 19...",3/4/65,Alive,Unknown
4,4,67,128029,128029,3271379,197,1594489505,9.781594e+12,Khaled Hosseini,2007.0,...,"['Afghanistan', 'Kabul', 'Herat']",['British Book Award for Best Read of the Year...,430,Khaled Hosseini,22,male,"Hosseini was born in Kabul, Afghanistan, in 19...",3/4/65,Alive,Unknown


In [5]:
books = X1["original_title"]      #creating the list of the books from the dataframe

In [6]:
(books)                         #viewing the book list


0                                       Angels & Demons 
1                                        Deception Point
2                                        The Lost Symbol
3                                       The Kite Runner 
4                               A Thousand Splendid Suns
5                               And The Mountains Echoed
6                                  The Devil Wears Prada
7                                       The Glass Castle
8                   Half Broke Horses: A True Life Novel
9      Freakonomics: A Rogue Economist Explores the H...
10                                              The Road
11                                No Country for Old Men
12                          The Memory Keeper's Daughter
13                                             Middlesex
14                                     The Marriage Plot
15                 The Sisterhood of the Traveling Pants
16                   The Second Summer of the Sisterhood
17     Girls in Pants: The Thir

In [7]:
books[0]

'Angels & Demons '

## Reddit

The following table is user to store data for the posts that have been posted regarding the books by the users.

In [88]:
bookList = []                            #List with the list of the books to extract data from reddit
postText = []                            #List of the texts posted about the book
postAuthor= []                           #List of the users that posted these posts
postID = []                              #List of the ID of each post
postEdited = []                          #List of the time if these posted were edited or if not, False.
postUps = []                             #List of the count of upvotes that these posts have recieved
postDowns = []                           #List of the count of downvotes recieved on each post
postLikes = []                           #List of the number of likes on these posts
postTime = []                            #List of the time when these posts were posted
c=0                                      #counter to print
count = 0                                #counter to limit to 10 posts
for book in books:
    print(c)
    c=c+1
    count = 0 
    for submission in reddit.subreddit('all').search(book):               #looping on the list
        bookList.append(book)                                             #appending to the lists from the submission object
        postText.append(submission.title)
        postAuthor.append(submission.author)
        postID.append(submission.id)
        postEdited.append(submission.edited)
        postUps.append(submission.ups)
        postDowns.append(submission.downs)
        postLikes.append(submission.likes)                              #using methods to modify time in the required format
        postTime.append(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(submission.created_utc)))
        count = count +1
        if (count == 10):
            break


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [89]:
print(len(bookList))
print(len(postText))
print(len(postAuthor))
print(len(postID))
print(len(postEdited))
print(len(postUps))
print(len(postDowns))
print(len(postLikes))

4959
4959
4959
4959
4959
4959
4959
4959


In [94]:
redditFile= pd.DataFrame({"Title":bookList,      #creating the dataframe from the lists that we have stored the data in 
                        "Post":postText,
                        "Post_Author":postAuthor,
                        "Post_ID" : postID,
                        "Post_Time":postTime,
                        "Post_Edited": postEdited,
                        "Up_Votes" : postUps,
                        "Down_Votes" : postDowns
                       })

In [95]:
redditFile.to_csv("reddit.csv")                  #creating a csv file to store the data from the dataframe

In [5]:
postFile = pd.read_csv("reddit.csv")            #reading the file that jas the names of all the users that have posted.
postAuthList = postFile["Post_Author"]

In [6]:
postAuthList                                    #list of the users that have posted regarding the books

0                  amchamp11
1                   Hristo21
2                _Ultimatum_
3                   dotFleur
4                  M0rph33u5
5                TheMegaGodz
6            fortifiedblonde
7         Machina_Ex_Machina
8                     Zondax
9                  JakzePoro
10           Wolfgirlkennedy
11                  boiuawsm
12             swishamane420
13                  drpauras
14                    Zoze13
15            birthingscapes
16            catherinsarris
17            voismitbispono
18                  Loki2121
19          scientistbybirth
20                     Smy24
21           vasishtsudhanva
22                  Dutch127
23             NoCloutFarOut
24             SnakebiteCafe
25        design-responsibly
26                   arktose
27               mannyledesm
28      Bing_Bong_the_Archer
29               6e_universe
                ...         
4929          BlackfishBlues
4930     YoureTwistinMyMelon
4931             RonPossible
4932     Youre

The folowing table is used to store data regarding posts that have been posted by the users

In [ ]:
userText = []                       #List of the text posted by the user.
userTime = []                       #List of the Time of the post, posted by the user
userName = []                       #List of the name of the user
userSubReddit = []                  #List of the SubReddits
i = 0                               # counter to display
count = 0
for au in postAuthList:             #for loop on the name of the user to create the redditor object
    print(i)
    i = i+1
    try:
        for userPost in reddit.redditor(str(au)).submissions.new(limit=7):   #creating the redditor object and limiting ot to 7 posts
            #print(comment.created_utc)
            userName.append(au)                                              #appending the reqiured values to our lists
            userText.append(userPost.title)
            userTime.append(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(userPost.created_utc)))
            userSubReddit.append(userPost.subreddit)
            #print("=====================")
            #print(userPost.title)
            #print(userPost.subreddit)
            #print(au)
            #print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(userPost.created_utc)))
    except:
            ka=1

In [9]:
print(len(userName))       #checking the length of the lists
print(len(userText))
print(len(userTime))
print(len(userSubReddit))

31370
31370
31370
31370


In [15]:
Reddit_user_posts =  pd.DataFrame({"Name":userName,      #creating the dataframe from the lists that we have stored the data in 
                        "Post":userText,
                        "Time":userTime,
                        "SubReddit" : userSubReddit
                       })

In [16]:
Reddit_user_posts.to_csv("Reddit_user_posts.csv")     #creating the .csv file to store the data

In [44]:
userName

[Redditor(name='Wolfgirlkennedy'),
 Redditor(name='Wolfgirlkennedy'),
 Redditor(name='Wolfgirlkennedy'),
 Redditor(name='Wolfgirlkennedy'),
 Redditor(name='Wolfgirlkennedy'),
 Redditor(name='boiuawsm'),
 Redditor(name='boiuawsm'),
 Redditor(name='boiuawsm'),
 Redditor(name='boiuawsm'),
 Redditor(name='boiuawsm'),
 Redditor(name='drpauras'),
 Redditor(name='drpauras'),
 Redditor(name='drpauras'),
 Redditor(name='drpauras'),
 Redditor(name='drpauras'),
 Redditor(name='swishamane420'),
 Redditor(name='swishamane420'),
 Redditor(name='swishamane420'),
 Redditor(name='swishamane420'),
 Redditor(name='swishamane420'),
 Redditor(name='Zoze13'),
 Redditor(name='Zoze13'),
 Redditor(name='Zoze13'),
 Redditor(name='Zoze13'),
 Redditor(name='Zoze13'),
 Redditor(name='birthingscapes'),
 Redditor(name='birthingscapes'),
 Redditor(name='birthingscapes'),
 Redditor(name='birthingscapes'),
 Redditor(name='birthingscapes'),
 Redditor(name='voismitbispono'),
 Redditor(name='Loki2121'),
 Redditor(name='Lo

## Twitter

In [9]:
consumer_key ='hGKg9zfaqHqPGlxvKdetGc95Q'                         #Setting up the connection with the twitter api 

consumer_secret ='IaLSKBGeFMnHZoIYETGuF80og4qo564xKsvyYZCu4Q2Mfc62Ic'

access_token = '1087797553121427456-fkyideSe6zR7pVaWbQAOWTahfHUMDX'

access_token_secret = 'JhQw3pmuR3GK8l1VE2lN1uvWAZhDMNn74ELEBa7c4zeC3'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

The following table is used to store twitter data of the authors

In [10]:
authorName = []                                   #List of the authors 
authorID = []                                     #List of the twitter IDs of the authors
authorCreatedAt=[]                                #Time at which the author ID was created
authorFavCount = []                               #List of the count the author has favorited 
authorFolCount = []                               #List of the number of followers the author has
authorDesc = []                                   #List of the Description of the author on their profile
authorFrndsCount = []                             #List of the number of people the author is following
authorStatusCount = []                            #List of the number of posts that the author has posted
authorTwitterList = ["JohnGrisham", "khaledhosseini","camphalfblood","AuthorDanBrown","AnnBrashares",
            "NicholasSparks","StephenKing","jodipicoult","JP_Books","harukimurakami_",
            "sarahdessen","jasperfforde","Tracy_Chevalier","Carl_Hiaasen","davidbaldacci",
            "nelsondemille","Gladwell","augusten","SalmanRushdie","SKenyonFans",
            "nealstephenson","Atul_Gawande","nickhornby","jk_rowling"]

for au in authorTwitterList:                     #Looping on the list of author Twitter IDs
    authorData = api.get_user(str(au))           #Appending the relevent information to our lists
    authorCreatedAt.append(authorData.created_at)
    authorFavCount.append(authorData.favourites_count)
    authorFolCount.append(authorData.followers_count)
    authorDesc.append(authorData.description)
    authorFrndsCount.append(authorData.friends_count)
    authorName.append(authorData.name)
    authorStatusCount.append(authorData.statuses_count)
    authorID.append(au)

In [12]:
Twitter_Author_Data= pd.DataFrame({"Name":authorName,      #creating the dataframe from the lists that we have stored the data in 
                        "ID":authorID,
                        "Created_AT":authorCreatedAt,
                        "Favourited" : authorFavCount,
                        "Followers": authorFolCount,
                        "Description" : authorDesc,
                        "Friend_Count" : authorFrndsCount,
                        "Tweets":authorStatusCount
                       })

In [13]:
print(len(authorName))
print(len(authorID))
print(len(authorCreatedAt))
print(len(authorFavCount))
print(len(authorFolCount))
print(len(authorDesc))
print(len(authorFrndsCount))
print(len(authorStatusCount))

24
24
24
24
24
24
24
24


In [14]:
Twitter_Author_Data.to_csv("Twitter_Auth_data.csv")   #saving the data in a .csv file

This table is used to store twitter data of the pubishers

In [15]:
publisherName = []                         #List of the publishers 
publisherID = []                           #List of the twitter IDs of the publishers
publisherCreatedAt=[]                      #Time at which the publisher ID was created
publisherFavCount = []                     #List of the count the publisher has favorited 
publisherFolCount = []                     #List of the number of followers the publisher has
publisherDesc = []                         #List of the Description of the publisher on their profile
publisherFrndsCount = []                   #List of the number of people the publisher is following
publisherStatusCount = []                  #List of the number of posts that the publisher has posted
publisher = ["Scholastic", "DoubledayUK","littlebrown","GrandCentralPub","DelacortePress",
            "Pocket_Books","AtriaBooks","VikingBooksUK","VintageCardBook","penguinrandom",
            "PicadorUSA","OUPAcademic","StMartinsPress","BerkleyPub","WmMorrowBooks",
            "MetropolitanBks","riverheadbooks","HarperCollins"]

for au in publisher:
    publisherData = api.get_user(str(au))
    publisherCreatedAt.append(publisherData.created_at)
    publisherFavCount.append(publisherData.favourites_count)
    publisherFolCount.append(publisherData.followers_count)
    publisherDesc.append(publisherData.description)
    publisherFrndsCount.append(publisherData.friends_count)
    publisherName.append(publisherData.name)
    publisherStatusCount.append(publisherData.statuses_count)
    publisherID.append(au)

In [16]:
Twitter_Publisher_Data= pd.DataFrame({"Name":publisherName,      #creating the dataframe from the lists that we have stored the data in 
                        "ID":publisherID,
                        "Created_AT":publisherCreatedAt,
                        "Favourited" : publisherFavCount,
                        "Followers": publisherFolCount,
                        "Description" : publisherDesc,
                        "Friend_Count" : publisherFrndsCount,
                        "Tweets":publisherStatusCount
                       })

In [17]:
Twitter_Publisher_Data.to_csv("Twitter_Pub_data.csv")      #creating the .csv file to store the data

The following table is used to store posts posted by the authors

In [18]:
screenNameAuthor = []                          #List of the IDs of the author who has posted the post.
userNameAuthor = []                            #List of the Name of the author who have posted this post.
textAuthor = []                                #List of the text of the posts.
hashtagsAuthor = []                            #List of the hashtags that have been posted by the authors.
favCountAuthor = []                            #List of the times that this post has been favourited
retCountAuthor = []                            #List of the times of the retweet on this post
timeAuthor = []                                #List of the time the posts have been posted
originalUserAuthor = []                        #List of the original user who has posted this post incase the tweet is a retweet
retweetedAuthor = []                           #List that holds if the tweet is a retweet or not
thislistAuthor = ["jk_rowling","JohnGrisham", "khaledhosseini","camphalfblood","AuthorDanBrown","AnnBrashares",
            "NicholasSparks","StephenKing","jodipicoult","JP_Books","harukimurakami_",
            "sarahdessen","jasperfforde","Tracy_Chevalier","Carl_Hiaasen","davidbaldacci",
            "nelsondemille","Gladwell","augusten","SalmanRushdie","SKenyonFans",
            "nealstephenson","Atul_Gawande","nickhornby"]
authors = []
for x in thislistAuthor:                        #looping on the authorlist
    for status in tweepy.Cursor(api.user_timeline, id=x, tweet_mode='extended').items(10):   #fetching the status 
        screenNameAuthor.append(status.user.screen_name)
        userNameAuthor.append(status.user.name)                  #appending the required fields
        #print(status.created_at)
        timeAuthor.append(status.created_at)

        #print(status.favorite_count)
        favCountAuthor.append(status.favorite_count)
        #print(status.retweet_count)
        retCountAuthor.append(status.retweet_count)
        hashtagsAuthor.append([e['text'] for e in status._json['entities']['hashtags']])
        try:                                                     #fetching the data for the retweets
            #print(status.retweeted_status.full_text)
            textAuthor.append(status.retweeted_status.full_text)
            #print(status.retweeted_status.user.name)
            originalUserAuthor.append(status.retweeted_status.user.name)
            retweetedAuthor.append("Y")
        except:
            textAuthor.append(status.full_text)
            originalUserAuthor.append("N/A")
            retweetedAuthor.append("N")

                                                                
Twitter_pd_author = pd.DataFrame({"Screen Name": screenNameAuthor,           #creating a dataframe from the lists of data
                        "Name":userNameAuthor,
                        "Tweet":textAuthor,
                        "HashTags":hashtagsAuthor,
                        "Favourites":favCountAuthor,
                        "Retweet Count":retCountAuthor,
                        "Time":timeAuthor,
                        "Original_User":originalUserAuthor,
                        "Retweeted":retweetedAuthor,
                       })

Twitter_pd_author                                            



,Screen Name,Name,Tweet,HashTags,Favourites,Retweet Count,Time,Original_User,Retweeted
0,jk_rowling,J.K. Rowling,@euanmccolm @chrisdeerin @Slowfinger45 @ropoem...,[],266,31,2019-03-01 11:08:46,N/A,N
1,jk_rowling,J.K. Rowling,"This week's column, on how when certain column...",[],0,533,2019-01-12 11:05:29,Caitlin Moran,Y
2,jk_rowling,J.K. Rowling,@UnitedHayze @DavidTaylar @Vinny I remain #Tea...,"[TeamDylan, TeamVinny]",103,7,2019-01-12 09:50:45,N/A,N
3,jk_rowling,J.K. Rowling,"""Dangerous and inflammatory talk about civil u...",[],0,1357,2019-01-12 09:36:03,BBC Politics,Y
4,jk_rowling,J.K. Rowling,"""Why would a man who claims to have dedicated ...",[],0,804,2019-01-12 09:11:30,The New European,Y
5,jk_rowling,J.K. Rowling,@scalzi Bring stuff you can trade. Bolts of si...,[],1716,44,2019-01-11 19:32:55,N/A,N
6,jk_rowling,J.K. Rowling,@Vinny I need you to know that David screamed ...,[teamVinny],312,25,2019-01-11 19:30:20,N/A,N
7,jk_rowling,J.K. Rowling,“Blame us. Blame Westminster. Do not blame Bru...,[],0,19789,2019-01-11 16:09:53,Channel 4 News,Y
8,jk_rowling,J.K. Rowling,‘The conclusion of populism is authoritarianis...,[],4208,988,2019-01-11 15:55:35,N/A,N
9,jk_rowling,J.K. Rowling,he's stroking that cock far too hard https://t...,[],0,1201,2019-01-11 15:14:58,The Museum of English Rural Life,Y


In [19]:
Twitter_pd_author.to_csv("Author_Tweets.csv")                  #creating a .csv file to store the data

The following table is used to store posts posted by the publishers

In [20]:
screenNamePublisher = []                                            #List of the IDs of the publisher who has posted the post. 
userNamePublisher = []                                              #List of the Name of the publisher who have posted this post.
textPublisher = []                                                  #List of the text of the posts.
hashtagsPublisher = []                                              #List of the hashtags that have been posted by the publisher.
favCountPublisher = []                                              #List of the times that this post has been favourited
retCountPublisher = []                                              #List of the times of the retweet on this post
timePublisher = []                                                  #List of the time the posts have been posted
originalUserPublisher = []                                          #List of the original user who has posted this post incase the tweet is a retweet
retweetedPublisher = []                                             #List that holds if the tweet is a retweet or not
thislistPublisher = ["Scholastic", "DoubledayUK","littlebrown","GrandCentralPub","DelacortePress",
            "Pocket_Books","AtriaBooks","VikingBooksUK","VintageCardBook","penguinrandom",
            "PicadorUSA","OUPAcademic","StMartinsPress","BerkleyPub","WmMorrowBooks",
            "MetropolitanBks","riverheadbooks","HarperCollins"]
publisher = []
for x in thislistPublisher:                                          #iterating the list of publishers
    for status in tweepy.Cursor(api.user_timeline, id=x, tweet_mode='extended').items(10):  #initializing the twitter object to fetch the data
        screenNamePublisher.append(status.user.screen_name)    #appending data to the fields
        userNamePublisher.append(status.user.name)
        #print(status.created_at)
        timePublisher.append(status.created_at)

        #print(status.favorite_count)
        favCountPublisher.append(status.favorite_count)
        #print(status.retweet_count)
        retCountPublisher.append(status.retweet_count)
        hashtagsPublisher.append([e['text'] for e in status._json['entities']['hashtags']])
        try:
            #print(status.retweeted_status.full_text)             #fetching the data for the retweets
            textPublisher.append(status.retweeted_status.full_text)
            #print(status.retweeted_status.user.name)
            originalUserPublisher.append(status.retweeted_status.user.name)
            retweetedPublisher.append("Y")
        except:
            textPublisher.append(status.full_text)
            originalUserPublisher.append("N/A")
            retweetedPublisher.append("N")


Twitter_pd_publisher = pd.DataFrame({"Screen Name": screenNamePublisher,      #creating a dataframe from the lists of data
                        "Name":userNamePublisher,
                        "Tweet":textPublisher,
                        "HashTags":hashtagsPublisher,
                        "Favourites":favCountPublisher,
                        "Retweet Count":retCountPublisher,
                        "Time":timePublisher,
                        "Original_User":originalUserPublisher,
                        "Retweeted":retweetedPublisher,
                       })

Twitter_pd_publisher

,Screen Name,Name,Tweet,HashTags,Favourites,Retweet Count,Time,Original_User,Retweeted
0,Scholastic,Scholastic,#OurCastleByTheSea by @theLucyStrange has rece...,[OurCastleByTheSea],3,0,2019-03-05 20:25:01,N/A,N
1,Scholastic,Scholastic,"Artificial Intelligence expert, @kaifulee spok...",[],3,0,2019-03-05 20:10:01,N/A,N
2,Scholastic,Scholastic,Our favorite baby sitters are headed to @netfl...,[],18,3,2019-03-05 18:16:00,N/A,N
3,Scholastic,Scholastic,"@mrchriswiacek We're sorry, Chris. At this tim...",[],0,0,2019-03-05 18:11:26,N/A,N
4,Scholastic,Scholastic,"@rayman_josh Hi, Josh! When we start having ou...",[],0,0,2019-03-05 18:00:02,N/A,N
5,Scholastic,Scholastic,Our @KidsPress Kid Reporters checked in from a...,[],7,1,2019-03-05 17:26:01,N/A,N
6,Scholastic,Scholastic,Calling all Book Fair organizers: Want to lear...,[],6,0,2019-03-05 16:50:41,N/A,N
7,Scholastic,Scholastic,We're still celebrating the more than 40 new b...,[],7,1,2019-03-05 16:08:01,N/A,N
8,Scholastic,Scholastic,"While reading brings us joy, science also tell...",[],71,40,2019-03-05 15:04:01,N/A,N
9,Scholastic,Scholastic,On our podcast episode all about narrative non...,[ScholasticFocus],3,0,2019-03-05 14:29:01,N/A,N


In [21]:
Twitter_pd_publisher.to_csv("Publisher_Tweets.csv")    #creating a .csv file to store the data

## Citations and References
https://praw.readthedocs.io/en/latest/getting_started/quick_start.html

http://docs.tweepy.org/en/v3.5.0/

## License

Copyright 2019 Akash Srivastava, Rashika Moza

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:
1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

Copyright 2019 Akash Srivastava, Rashika Moza

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


